In [50]:
import os
import time

In [51]:
#Size of the file
os.path.getsize("C:/Users/Shreya/Downloads/CovidDeaths.csv")

28938085

# Read in the data with Pandas

In [52]:
import pandas as pd
start = time.time()
df = pd.read_csv('C:/Users/Shreya/Downloads/CovidDeaths.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  0.7740767002105713 sec


# Read in the data with Dask

In [53]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('C:/Users/Shreya/Downloads/CovidDeaths.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.01208353042602539 sec


# Read in the data with Modin and Ray

In [54]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('C:/Users/Shreya/Downloads/CovidDeaths.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2023-01-11 00:22:07,148	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Read csv with modin and ray:  2.536426305770874 sec


Here Dask has the least reading time of 0.012 sec and is better compared with Pandas, Modin and Ray while working with large size of data

In [55]:
from dask import dataframe as dd
df = dd.read_csv('C:/Users/Shreya/Downloads/CovidDeaths.csv',delimiter=',')

In [56]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 26 entries, iso_code to weekly_hosp_admissions_per_million
dtypes: object(4), float64(20), int64(2)

In [57]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [58]:
#To remove white space from columns
df.columns = df.columns.str.replace(' ', '')

In [59]:
data=df.columns
data

Index(['iso_code', 'continent', 'location', 'date', 'population_density',
       'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths',
       'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million'],
      dtype='object')

In [60]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re



def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0
                                                                

Overwriting testutility.py


In [61]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: test_data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - iso_code 
    - continent 
    - location 
    - date 
    - population_density
    - total_cases   
    - new_cases 
    - new_cases_smoothed 
    - total_deaths
    - new_deaths
    - new_deaths_smoothed 
    - total_cases_per_million
    - new_cases_per_million
    - new_cases_smoothed_per_million
    - total_deaths_per_million 
    - new_deaths_per_million
    - new_deaths_smoothed_per_million 
    - reproduction_rate 
    - icu_patients
    - icu_patients_per_million
    - hosp_patients
    - hosp_patients_per_million
    - weekly_icu_admissions
    - weekly_icu_admissions_per_million
    - weekly_hosp_admissions
    - weekly_hosp_admissions_per_million


Overwriting file.yaml


In [62]:
# Read config file
import testutility as util
config_data = util.read_config_file("file.yaml")

In [63]:
#data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'testfile',
 'file_name': 'test_data',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['iso_code',
  'continent',
  'location',
  'date',
  'population_density',
  'total_cases',
  'new_cases',
  'new_cases_smoothed',
  'total_deaths',
  'new_deaths',
  'new_deaths_smoothed',
  'total_cases_per_million',
  'new_cases_per_million',
  'new_cases_smoothed_per_million',
  'total_deaths_per_million',
  'new_deaths_per_million',
  'new_deaths_smoothed_per_million',
  'reproduction_rate',
  'icu_patients',
  'icu_patients_per_million',
  'hosp_patients',
  'hosp_patients_per_million',
  'weekly_icu_admissions',
  'weekly_icu_admissions_per_million',
  'weekly_hosp_admissions',
  'weekly_hosp_admissions_per_million']}

In [47]:
config_data['inbound_delimiter']

','

In [65]:
# Normal reading process of the file
import pandas as pd
df_sample = pd.read_csv("C:/Users/Shreya/Downloads/CovidDeaths.csv",delimiter=',')
df_sample.head()

,iso_code,continent,location,date,population_density,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million
0,AFG,Asia,Afghanistan,2020-02-24,54.422,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# read the file using config file
file_type = config_data['file_type']
source_file = "C:/Users/Shreya/" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,iso_code,continent,location,date,population_density,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million
0,AFG,Asia,Afghanistan,2020-02-24,54.422,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
#validating the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [76]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['iso_code', 'continent', 'location', 'date', 'population_density',
       'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths',
       'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million'],
      dtype='object')
columns of YAML are: ['iso_code', 'continent', 'location', 'date', 'population_density', 'total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million', 'new_cases_per_million', 'new_cases_smoothed_per_million', 'total_deaths_per_million

In [77]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")

column name and column length validation passed
col validation passed


In [78]:
pd.read_csv(r"C:\Users\Shreya\test_data.csv")

,iso_code,continent,location,date,population_density,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,...,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million
0,AFG,Asia,Afghanistan,2020-02-24,54.422,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-02-25,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-02-26,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-02-27,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-02-28,54.422,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248155,ZWE,Africa,Zimbabwe,2023-01-02,42.729,259981.0,0.0,0.0,5637.0,0.0,...,0.0,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248156,ZWE,Africa,Zimbabwe,2023-01-03,42.729,259981.0,0.0,0.0,5637.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248157,ZWE,Africa,Zimbabwe,2023-01-04,42.729,259981.0,0.0,0.0,5637.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248158,ZWE,Africa,Zimbabwe,2023-01-05,42.729,259981.0,0.0,0.0,5637.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
import csv 
with open('C:/Users/Shreya/test_data.csv') as fin:
    with open('C:/Users/Shreya/test_data1.csv', 'w', newline='') as fout:
        reader = csv.DictReader(fin, delimiter=',')
        writer = csv.DictWriter(fout, reader.fieldnames, delimiter='|')
        writer.writeheader()
        writer.writerows(reader)

In [89]:
df1 = pd.read_csv('C:/Users/Shreya/test_data1.csv')
df1

,iso_code|continent|location|date|population_density|total_cases|new_cases|new_cases_smoothed|total_deaths|new_deaths|new_deaths_smoothed|total_cases_per_million|new_cases_per_million|new_cases_smoothed_per_million|total_deaths_per_million|new_deaths_per_million|new_deaths_smoothed_per_million|reproduction_rate|icu_patients|icu_patients_per_million|hosp_patients|hosp_patients_per_million|weekly_icu_admissions|weekly_icu_admissions_per_million|weekly_hosp_admissions|weekly_hosp_admissions_per_million
0,AFG|Asia|Afghanistan|2020-02-24|54.422|5|5||||...
1,AFG|Asia|Afghanistan|2020-02-25|54.422|5|0||||...
2,AFG|Asia|Afghanistan|2020-02-26|54.422|5|0||||...
3,AFG|Asia|Afghanistan|2020-02-27|54.422|5|0||||...
4,AFG|Asia|Afghanistan|2020-02-28|54.422|5|0||||...
...,...
248155,ZWE|Africa|Zimbabwe|2023-01-02|42.729|259981|0...
248156,ZWE|Africa|Zimbabwe|2023-01-03|42.729|259981|0...
248157,ZWE|Africa|Zimbabwe|2023-01-04|42.729|259981|0...
248158,ZWE|Africa|Zimbabwe|2023-01-05|42.729|259981|0...


In [90]:
df1.to_csv('dfsavename.csv.gz', compression='gzip')
df1 = pd.read_csv('dfsavename.csv.gz', compression='gzip')

In [91]:
df1.describe()

,Unnamed: 0
count,248160.000000
mean,124079.500000
std,71637.765738
min,0.000000
25%,62039.750000
50%,124079.500000
75%,186119.250000
max,248159.000000


In [92]:
# computing number of rows
rows = len(df1.axes[0])
 
# computing number of columns
cols = len(df1.axes[1])
 
print(df1)
print("Number of Rows: ", rows)
print("Number of Columns: ", cols)

        Unnamed: 0  \
0                0   
1                1   
2                2   
3                3   
4                4   
...            ...   
248155      248155   
248156      248156   
248157      248157   
248158      248158   
248159      248159   

       iso_code|continent|location|date|population_density|total_cases|new_cases|new_cases_smoothed|total_deaths|new_deaths|new_deaths_smoothed|total_cases_per_million|new_cases_per_million|new_cases_smoothed_per_million|total_deaths_per_million|new_deaths_per_million|new_deaths_smoothed_per_million|reproduction_rate|icu_patients|icu_patients_per_million|hosp_patients|hosp_patients_per_million|weekly_icu_admissions|weekly_icu_admissions_per_million|weekly_hosp_admissions|weekly_hosp_admissions_per_million  
0       AFG|Asia|Afghanistan|2020-02-24|54.422|5|5||||...                                                                                                                                                                     

In [93]:
#obtaining file size
import os
os.path.getsize("C:/Users/Shreya/test_data1.csv")

28938085